In [ ]:
#IMDB

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
from sklearn import svm

In [48]:
#load reviews
train = pd.read_csv('data/IMDB_train.csv')
test = pd.read_csv('data/IMDB_test.csv')
valid = pd.read_csv('data/IMDB_dev.csv')

#add review length to data framce
#yelp['text length'] = yelp['text'].str.len()

#sample for testing purppses
#yelp = yelp.sample(frac=.5)

X_train = train['text']
y_train = train['rating']

X_test = test['text']
y_test= test['rating']

X_valid = valid['text']
y_valid = valid['rating']

X_full = X_train + X_valid

In [49]:
max_vocab = 50000
bow_transformer = CountVectorizer(max_features=max_vocab, binary=True, ngram_range=(1,3)).fit(X_full.values.astype('U'))

In [50]:
print(X_train.shape)
print(X_test.shape)
print(X_valid.shape)
train = []
valid = []
test = []

X_train = bow_transformer.transform(X_train.values.astype('U'))
X_valid = bow_transformer.transform(X_valid.values.astype('U'))
X_test = bow_transformer.transform(X_test.values.astype('U'))

(280593,)
(34029,)
(33793,)


In [51]:
from sklearn.naive_bayes import MultinomialNB
import scipy.sparse as sp

X_full = sp.vstack((X_train , X_valid))
y_full = np.append(y_train, y_valid)

nb = MultinomialNB()
nb.fit(X_full, y_full)
y_hat = nb.predict(X_test)

print(accuracy_score(y_test, y_hat))

0.35599047870933614


In [ ]:
print(type (y_train[0]))

In [52]:
#SVM
def linSVM(parameters, X, T, fold) : 
    n_folds = 5
    svc = LinearSVC(multi_class='ovr', dual=False)
    svc_cv = GridSearchCV(svc, parameters, cv=fold, scoring="accuracy", refit=False)
    svc_cv.fit(X, T)
    scores = svc_cv.cv_results_['mean_test_score']
    params = svc_cv.cv_results_['params']
    print('scores:',scores)
    #print('params:',params)
    return svc_cv.best_params_

def SVM(parameters, X, T, fold) : 
    n_folds = 5
    svmm = svm.SVC(kernel='linear')
    svm_cv = GridSearchCV(svmm, parameters, cv=fold, scoring="accuracy", refit=False)
    svm_cv.fit(X, T)
    scores = svm_cv.cv_results_['mean_test_score']
    params = svm_cv.cv_results_['params']
    print('scores:',scores)
    print('params:',params)
    return svm_cv.best_params_

def MNB(parameters, X, T, fold) : 
    n_folds = 5
    mnb = MultinomialNB(fit_prior=True)
    mnb_cv = GridSearchCV(mnb, parameters, cv=fold, scoring="accuracy", refit=False)
    mnb_cv.fit(X, T)
    scores = mnb_cv.cv_results_['mean_test_score']
    params = mnb_cv.cv_results_['params']
    print('scores:',scores)
   #print('params:',params)
    return mnb_cv.best_params_


#Predefined Split for CV
my_test_fold = []
for i in range(X_train.shape[0]):
    my_test_fold.append(-1)
for i in range(X_valid.shape[0]):
    my_test_fold.append(0)

fold = PredefinedSplit(test_fold=my_test_fold)
CV_split_T = np.append(y_train, y_valid)

import scipy.sparse as sp
CV_split_X = sp.vstack((X_train , X_valid))  # NOT np.vstack


print(CV_split_T.shape)
print(CV_split_X.shape)


(314386,)
(314386, 50000)


In [ ]:
# Cs = [ .0001, 0.001, 0.01, 0.1]


param_grid = {'C': Cs}
bestParams = linSVM(param_grid, CV_split_X, CV_split_T, fold)



svc = LinearSVC(C=bestParams['C', multi_class='ovr', dual=False)       
svc.fit(X_train, y_train)
y_hat = svc.predict(X_test)
print(accuracy_score(y_test, y_hat))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/zmq/eventloop/zmqstream.py", line 432, in _run_callback
    callback(*args, **kwargs)
  File "/anaconda3/lib/python3.6/site-packages/tornado/stack_context.py", line 276, in null_wrapper
    return fn(*args, **kwargs)
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 233, in dispatch_shell
    handler(stream, idents, msg)
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 421, in execute_request
    self._abort_queues()
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 636, in _abort_queues
    self._

In [ ]:
#MNB
alphas = [ 0.0001, 0.001, 0.01, 0.1, 1, 10, 100]


param_grid = {'alpha': alphas}

bestParams = MNB(param_grid, CV_split_X, CV_split_T, fold)

nb = MultinomialNB(alpha=bestParams['alpha'], fit_prior=True)
nb.fit(X_train, y_train)
y_hat = nb.predict(X_test)

print(accuracy_score(y_test, y_hat))

In [ ]:
print(confusion_matrix(y_test, y_hat))
print('\n')
print(classification_report(y_test, y_hat))

print(accuracy_score(y_test, y_hat))